## Escenarios de migración interna

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [2]:
prodSim = sm.load('modelos/restringido_origen__siMVO_v1.pkl')
prodSim

In [3]:
# carga datos diádicos
dd_deptos = pd.read_csv('tablas/dd_deptos.csv')
dd_deptos.loc[dd_deptos.largo_limite.isna(), 'largo_limite'] = 0.0001
print(dd_deptos.shape)
dd_deptos.head(2)

(342, 20)


,cod,depto_origen,depto_destino,personas_mig,personas_mig_100k,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,pbi_porcen_des,largo_limite,dummy_limit,empresas_origen,empresas_destino,log_pbi_destino,log_dist,nom_depto_orig,nom_depto_des
0,102,1,2,914,891,1318755,73377,610726,465848031,50.3,14214980,1.5,0.0001,False,3124,55,16.469807,13.322404,MONTEVIDEO,ARTIGAS
1,103,1,3,33127,0,1318755,520173,22544,465848031,50.3,86306492,9.3,60107.0000,True,3124,563,18.273415,10.023224,MONTEVIDEO,CANELONES


In [4]:
dd_deptos['log_pbi_destino'] = dd_deptos['log_pbi_destino']*1.005
dd_deptos.head(2)

,cod,depto_origen,depto_destino,personas_mig,personas_mig_100k,pob_origen,pob_destino,dist,pbi_origen,pbi_porcen_ori,pbi_destino,pbi_porcen_des,largo_limite,dummy_limit,empresas_origen,empresas_destino,log_pbi_destino,log_dist,nom_depto_orig,nom_depto_des
0,102,1,2,914,891,1318755,73377,610726,465848031,50.3,14214980,1.5,0.0001,False,3124,55,16.552156,13.322404,MONTEVIDEO,ARTIGAS
1,103,1,3,33127,0,1318755,520173,22544,465848031,50.3,86306492,9.3,60107.0000,True,3124,563,18.364782,10.023224,MONTEVIDEO,CANELONES


In [5]:
pred = prodSim.get_prediction(dd_deptos)

In [6]:
dd_deptos['prodsimest'] = pred.summary_frame()['mean'].round(0).astype(int)

In [7]:
# matriz de los valores estimados
pd.pivot_table(dd_deptos,
values='prodsimest',
index ='depto_origen',
columns='depto_destino',
fill_value=0,
aggfunc=sum,
margins=True,
margins_name='Total')

depto_destino,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,Total
depto_origen,,,,,,,,,,,,,,,,,,,,
1,0,823,35290,1104,3859,899,453,1463,1193,4901,1911,1747,1298,1233,1718,3779,1695,1202,743,65311
2,2520,0,571,94,270,67,33,94,79,315,169,145,134,90,742,152,132,109,58,5774
3,14983,79,0,107,364,87,44,494,337,1094,184,168,125,120,165,673,163,116,72,19375
4,2095,58,479,0,214,190,27,79,70,274,122,107,296,81,116,124,101,297,254,4984
5,2445,56,544,71,0,60,38,87,70,282,136,129,87,76,120,481,506,81,47,5316
6,1663,40,379,186,175,0,84,275,52,201,95,309,65,56,85,102,83,273,76,4199
7,723,17,165,22,95,72,0,81,22,85,43,47,27,23,37,145,140,26,15,1785
8,1604,34,1275,45,151,163,56,0,175,186,79,71,53,49,70,304,68,50,84,4517
9,1296,28,861,40,120,30,15,173,0,698,63,57,44,144,57,74,55,41,104,3900
